In [1]:
import warnings; warnings.simplefilter('ignore')

In [3]:
from growing_instability_lib import *
# Start mem 476 MB

from gensim.corpora import Dictionary
from gensim.models import tfidfmodel
from gensim.similarities import MatrixSimilarity
from gensim.models.word2vec import LineSentence
from gensim import corpora

In [4]:
lsn = LineSentence('./corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv')

In [5]:
# %%time
# dictionary = Dictionary(lsn)
# dictionary.id2token = {j: i for i, j in dictionary.token2id.items()}
# dictionary.save(lsn.source + '.dictionary')

# CPU times: user 10min 31s, sys: 2.18 s, total: 10min 34s
# Wall time: 10min 34s

In [6]:
# %%time
# corpus = [dictionary.doc2bow(i) for i in lsn]

# CPU times: user 10min 59s, sys: 3min 26s, total: 14min 26s
# Wall time: 31min 6s

In [7]:
# corpora.MmCorpus.serialize(lsn.source + '.corpora', corpus)

In [8]:
dictionary = Dictionary.load(lsn.source + '.dictionary')

In [9]:
def _corpus_generator(lsn):
    for l in lsn:
        yield dictionary.doc2bow(l)

corpus_generator = _corpus_generator(lsn)

In [10]:
# %%time
# tfidf = tfidfmodel.TfidfModel(corpus_generator)
# tfidf.save(lsn.source + '.tfidf')

# CPU times: user 2min 8s, sys: 26.1 s, total: 2min 34s
# Wall time: 7min 28s

In [11]:
tfidf = tfidfmodel.TfidfModel.load(lsn.source + '.tfidf')

In [ ]:
lsi = lsimodel.Lsimodel.load(lsn.source + '.lsi')

In [12]:
tfidf[dictionary.doc2bow(['zika'])]

[(215720, 1.0)]

In [13]:
def transform_tfidf_word2vec(tokens, stopwords=[]):
#     global wvmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    wv_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords and w in wvmodel.wv.vocab)]
    ).map(
        lambda x: tfidf[dictionary.doc2bow(x)]
    ).map(
        lambda x: np.array([wvmodel[dictionary.id2token[id]] * w for id, w in x]).mean(axis=0) if len(x) > 0 else np.nan
    )

    return wv_feature_vec


def transform_tfidf_fasttext(tokens, stopwords=[]):
#     global fsmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    fs_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    ).map(
        lambda x: tfidf[dictionary.doc2bow(x)]
    ).map(
        lambda x: np.array([np.array(fsmodel[dictionary.id2token[id]]) * w for id, w in x]).mean(axis=0) if len(x) > 0 else np.nan
    )

    return fs_feature_vec


def transform_tfidf_lsi(tokens, stopwords=[]):
#     global fsmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    lsi_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    ).map(
        lambda x: lsi[tfidf[dictionary.doc2bow(x)]] if len(x) > 0 else np.nan
    )

    return lsi_feature_vec


def transform_fasttext(tokens, stopwords=[]):
#     global fsmodel
    # This requires fsmodel to be present in the namespace.
    fs_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    ).map(lambda x: np.array([fsmodel[w] for w in x]).mean(axis=0) if len(x) > 0 else np.nan)

    return fs_feature_vec


def transform_unsupervised_sentiment_neuron(tokens, stopwords=[]):
    # This requires fsmodel to be present in the namespace.
    
    usn_feature_vec = usnmodel.transform(tokens)

    # usn_feature_vec = tokens.map(
    #     lambda x: [w for w in x.split() if (w not in stopwords)]
    # ).map(lambda x: np.array([usnmodel[w] for w in x]).mean(axis=0) if len(x) > 0 else np.nan)

    return usn_feature_vec


def transform_word2vec(tokens, stopwords=[]):
#     global wvmodel
    # This requires wvmodel to be present in the namespace.
    wv_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords and w in wvmodel.wv.vocab)]
    ).map(lambda x: np.array([wvmodel[w] for w in x]).mean(axis=0) if len(x) > 0 else np.nan)

    return wv_feature_vec


def parallel_generate_word_vectors(samp, transformer, stopwords, batch, num_proc):
    with Parallel(n_jobs=num_proc) as parallel:
        dataset = []
        is_break = False
        i = 0

        while not is_break:
            payload = []

            for j in xrange(num_proc):
                t_df = samp[(i + j) * batch: (i + 1 + j) * batch]

                if t_df.empty:
                    is_break = True
                    continue

                payload.append(
                    delayed(transformer)(
                        t_df, stopwords
                    )
                )

            print('Current batch in main thread: {}'.format((i + j) * batch))

            if payload:
                results = parallel(payload)
                dataset.extend(results)
                i += num_proc

    return pd.concat(dataset)


In [14]:
wvmodel = Word2Vec.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.w2v.model')

In [15]:
fsmodel = fasttext.load_model('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.fasttext.model.bin')

In [16]:
%%time
_word2idx = {}

def word2idx(word):
    ix = len(_word2idx)
    _word2idx[word] = _word2idx.get(word, ix)
    return _word2idx[word]

def generate_training_word_vectors(stopwords=set()):
    processed_tname = set()
    nan_rows = set()
    target_columns = sorted(topics)
    wvec_trainingX = pd.DataFrame()
    fvec_trainingX = pd.DataFrame()
    tfidf_wvec_trainingX = pd.DataFrame()
    tfidf_fvec_trainingX = pd.DataFrame()
    tfidf_lsi_trainingX = pd.DataFrame()
    word2idx_trainingX = pd.DataFrame()
    trainingY = pd.DataFrame()
    num_proc = 8

    for ix, tname in enumerate(glob.iglob('../data/TrainingData/*_TrainingData.json')):
        print('Processing {}. {}...'.format(ix + 1, tname))

        if tname in processed_tname or ix > 100:
            continue

        ds = parse_training_data_with_valid_topics(tname, topics=topics)
        ds = pd.DataFrame(ds).T

        for t in set(topics).difference(ds.columns):
            ds[t] = np.nan

        print('Done parsing {} docs with valid topics...'.format(ds.shape[0]))

        df_tokens = transform_text(ds)
        batch = min(df_tokens.shape[0] / num_proc, 2000)

        print('Computing fs features...')
        fvec = parallel_generate_word_vectors(df_tokens, transform_fasttext, stopwords=stopwords, batch=batch, num_proc=num_proc)

        print('Computing wv features...')
        wvec = parallel_generate_word_vectors(df_tokens, transform_word2vec, stopwords=stopwords, batch=batch, num_proc=num_proc)

        print('Computing tfidf fs features...')
        tfidf_fvec = parallel_generate_word_vectors(df_tokens, transform_tfidf_fasttext, stopwords=stopwords, batch=batch, num_proc=num_proc)

        print('Computing tfidf wv features...')
        tfidf_wvec = parallel_generate_word_vectors(df_tokens, transform_tfidf_word2vec, stopwords=stopwords, batch=batch, num_proc=num_proc)

        print('Computing tfidf lsi features...')
        tfidf_lsi = parallel_generate_word_vectors(df_tokens, transform_tfidf_lsi, stopwords=stopwords, batch=batch, num_proc=num_proc)
        
        nulls = set()

        nulls.update(fvec[fvec.isnull()].index.values)
        nulls.update(wvec[wvec.isnull()].index.values)
        nulls.update(tfidf_fvec[tfidf_fvec.isnull()].index.values)
        nulls.update(tfidf_wvec[tfidf_wvec.isnull()].index.values)
        nulls.update(tfidf_lsi[tfidf_lsi.isnull()].index.values)

        
        print 'Droping {} NaN samples...'.format(len(nulls))

        index = wvec.index.difference(nulls)
        wvec = wvec.ix[index]
        fvec = fvec.ix[index]
        tfidf_wvec = tfidf_wvec.ix[index]
        tfidf_fvec = tfidf_fvec.ix[index]
        tfidf_lsi = tfidf_lsi.ix[index]
        
        if wvec_trainingX.empty:
            wvec_trainingX = wvec
        else:
            wvec_trainingX = wvec_trainingX.append(wvec)

        if fvec_trainingX.empty:
            fvec_trainingX = fvec
        else:
            fvec_trainingX = fvec_trainingX.append(fvec)

        if tfidf_wvec_trainingX.empty:
            tfidf_wvec_trainingX = tfidf_wvec
        else:
            tfidf_wvec_trainingX = tfidf_wvec_trainingX.append(tfidf_wvec)

        if tfidf_fvec_trainingX.empty:
            tfidf_fvec_trainingX = tfidf_fvec
        else:
            tfidf_fvec_trainingX = tfidf_fvec_trainingX.append(tfidf_fvec)

        if tfidf_lsi_trainingX.empty:
            tfidf_lsi_trainingX = tfidf_lsi
        else:
            tfidf_lsi_trainingX = tfidf_lsi_trainingX.append(tfidf_lsi)

        print('Mapping word indices...')
        word_indices = df_tokens.ix[index].map(lambda x: [word2idx(i) for i in x.split()])

        if word2idx_trainingX.empty:
            word2idx_trainingX = word_indices
        else:
            word2idx_trainingX = word2idx_trainingX.append(word_indices)

        print('Current training size is {}...'.format(word2idx_trainingX.shape[0]))

        targets = ds[target_columns].ix[index]  # .fillna(0).as_matrix()

        if trainingY.empty:
            trainingY = targets
        else:
            trainingY = trainingY.append(targets)

        print('Completed extracting data from {}...'.format(tname))
        print
        processed_tname.add(tname)

    wvec_trainingX.to_hdf('training_data_wv_fs_no_stopwords.hdf', 'wvec_trainingX')
    fvec_trainingX.to_hdf('training_data_wv_fs_no_stopwords.hdf', 'fvec_trainingX')
    
    tfidf_wvec_trainingX.to_hdf('training_data_wv_fs_no_stopwords.hdf', 'tfidf_wvec_trainingX')
    tfidf_fvec_trainingX.to_hdf('training_data_wv_fs_no_stopwords.hdf', 'tfidf_fvec_trainingX')
    tfidf_lsi_trainingX.to_hdf('training_data_wv_fs_no_stopwords.hdf', 'tfidf_lsi_trainingX')

    word2idx_trainingX.to_hdf('training_data_wv_fs_no_stopwords.hdf', 'word2idx_trainingX')

    pd.Series(_word2idx).to_hdf('training_data_wv_fs_no_stopwords.hdf', '_word2idx')
    trainingY.to_hdf('training_data_wv_fs_no_stopwords.hdf', 'trainingY')

# CPU times: user 3h 11min 14s, sys: 2min 32s, total: 3h 13min 47s
# Wall time: 3h 12min 7s

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.87 µs


In [7]:
%%time
# start: 5395 MB
generate_training_word_vectors()

Processing 1. ../data/TrainingData/2003b_TrainingData.json...
Done parsing 6786 docs with valid topics...
Computing fs features...
Current batch in main thread: 5936
Current batch in main thread: 12720
Computing wv features...
Current batch in main thread: 5936
Current batch in main thread: 12720
Droping 0 NaN samples...
Mapping word indices...
Current training size is 6786...
Completed extracting data from ../data/TrainingData/2003b_TrainingData.json...

Processing 2. ../data/TrainingData/2004b_TrainingData.json...
Done parsing 7863 docs with valid topics...
Computing fs features...
Current batch in main thread: 6874
Current batch in main thread: 14730
Computing wv features...
Current batch in main thread: 6874
Current batch in main thread: 14730
Droping 0 NaN samples...
Mapping word indices...
Current training size is 14649...
Completed extracting data from ../data/TrainingData/2004b_TrainingData.json...

Processing 3. ../data/TrainingData/2005a_TrainingData.json...
Done parsing 6756

Completed extracting data from ../data/TrainingData/2001b_TrainingData.json...

Processing 19. ../data/TrainingData/2010b_TrainingData.json...
Done parsing 9561 docs with valid topics...
Computing fs features...
Current batch in main thread: 8365
Current batch in main thread: 17925
Computing wv features...
Current batch in main thread: 8365
Current batch in main thread: 17925
Droping 4 NaN samples...
Mapping word indices...
Current training size is 132578...
Completed extracting data from ../data/TrainingData/2010b_TrainingData.json...

Processing 20. ../data/TrainingData/2002b_TrainingData.json...
Done parsing 6180 docs with valid topics...
Computing fs features...
Current batch in main thread: 5404
Current batch in main thread: 11580
Computing wv features...
Current batch in main thread: 5404
Current batch in main thread: 11580
Droping 0 NaN samples...
Mapping word indices...
Current training size is 138758...
Completed extracting data from ../data/TrainingData/2002b_TrainingData.jso

In [17]:
%%time
# start: 5395 MB
generate_training_word_vectors()

Processing 1. ../data/TrainingData/2003b_TrainingData.json...
Done parsing 6786 docs with valid topics...
Computing fs features...
Current batch in main thread: 5936
Current batch in main thread: 12720
Computing wv features...
Current batch in main thread: 5936
Current batch in main thread: 12720
Computing tfidf fs features...
Current batch in main thread: 5936
Current batch in main thread: 12720
Computing tfidf wv features...
Current batch in main thread: 5936
Current batch in main thread: 12720
Droping 0 NaN samples...
Mapping word indices...
Current training size is 6786...
Completed extracting data from ../data/TrainingData/2003b_TrainingData.json...

Processing 2. ../data/TrainingData/2004b_TrainingData.json...
Done parsing 7863 docs with valid topics...
Computing fs features...
Current batch in main thread: 6874
Current batch in main thread: 14730
Computing wv features...
Current batch in main thread: 6874
Current batch in main thread: 14730
Computing tfidf fs features...
Current

Computing tfidf wv features...
Current batch in main thread: 7007
Current batch in main thread: 15015
Droping 0 NaN samples...
Mapping word indices...
Current training size is 178526...
Completed extracting data from ../data/TrainingData/2007b_TrainingData.json...

Processing 26. ../data/TrainingData/2000a_TrainingData.json...
Done parsing 2044 docs with valid topics...
Computing fs features...
Current batch in main thread: 1785
Current batch in main thread: 3825
Computing wv features...
Current batch in main thread: 1785
Current batch in main thread: 3825
Computing tfidf fs features...
Current batch in main thread: 1785
Current batch in main thread: 3825
Computing tfidf wv features...
Current batch in main thread: 1785
Current batch in main thread: 3825
Droping 0 NaN samples...
Mapping word indices...
Current training size is 180570...
Completed extracting data from ../data/TrainingData/2000a_TrainingData.json...

Processing 27. ../data/TrainingData/2011a_TrainingData.json...
Done par

In [125]:
trainingY = pd.read_hdf('training_data_wv_fs_no_stopwords', 'trainingY_fasttext')

In [142]:
tty = trainingY.copy()
freqs = tty.sum().sort_values(ascending=False)

In [137]:
freqs.head()

ukcrime        20513.0
iraq           20344.0
religion       16951.0
immigration    15296.0
london         15129.0
dtype: float64

In [130]:
freqs.describe()

count      146.000000
mean      2254.082192
std       3955.853032
min          3.000000
25%        212.750000
50%        630.000000
75%       2052.750000
max      20513.000000
dtype: float64

In [131]:
indices = resample_data(tty, upsample=100, maxsample=400)

In [143]:
tty.ix[indices].sum().sort_values(ascending=False)

humanrights                    2203.0
military                       2060.0
iraq                           1986.0
ukcrime                        1962.0
uksecurity                     1682.0
afghanistan                    1663.0
syria                          1595.0
immigration                    1546.0
police                         1506.0
london                         1435.0
unitednations                  1331.0
israel                         1311.0
religion                       1219.0
protest                        1198.0
terrorism                      1176.0
france                         1133.0
alqaida                        1131.0
defence                        1056.0
arabandmiddleeastprotests      1056.0
criminaljustice                 980.0
russia                          902.0
refugees                        897.0
aid                             893.0
india                           892.0
hacking                         876.0
surveillance                    869.0
naturaldisas

In [147]:
tty['year'] = tty.index.map(lambda x: re.findall('^([0-9]{4}).*', x)[0])

In [170]:
time_distribution = tty[freqs.head(len(freqs) - 14).index.union(['year'])].groupby('year').sum()

In [183]:
%matplotlib inline

time_distribution #.plot()

,ukcrime,iraq,religion,immigration,london,economy,israel,transport,afghanistan,france,...,internallydisplacedpeople,sydneysiege,cobra,encryption,southchinasea,darknet,hashtags,values,debate,australianguncontrol
year,,,,,,,,,,,,,,,,,,,,,
1999,246,59,172,42,177,18,91,182,35,NaN,...,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2000,335,67,316,263,374,37,595,538,8,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2001,481,122,329,738,347,326,868,770,2620,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2002,1008,1168,594,1309,391,678,1243,914,687,408.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,1346,6098,593,1677,441,902,797,716,167,592.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,1650,3066,777,2472,275,1206,882,771,199,659.0,...,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,NaN
2005,1495,1769,1072,1815,252,960,497,499,156,626.0,...,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
2006,1731,1576,1211,1709,345,837,1133,767,374,541.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2007,2024,2039,1175,2034,448,967,849,1049,452,805.0,...,NaN,NaN,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [176]:
unique_topic_articles = tty[freqs.index].sum(axis=1)
unique_topic_articles = unique_topic_articles[unique_topic_articles == 1]

In [182]:
tty.ix[unique_topic_articles.index][freqs.head(len(freqs) - 14).index.union(['year'])].groupby('year').sum()

,ukcrime,iraq,religion,immigration,london,economy,israel,transport,afghanistan,france,...,internallydisplacedpeople,sydneysiege,cobra,encryption,southchinasea,darknet,hashtags,values,debate,australianguncontrol
year,,,,,,,,,,,,,,,,,,,,,
1999,214,51,169,42,175,18,88,178,28,NaN,...,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2000,310,57,293,60,347,37,509,511,4,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2001,383,91,228,338,197,301,728,660,1436,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2002,621,869,467,696,264,633,1093,784,391,282.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,868,4897,469,674,279,827,699,573,109,452.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,1034,2420,578,1534,171,1087,740,587,151,481.0,...,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN,NaN
2005,1079,1342,840,1006,166,894,442,406,103,552.0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
2006,1111,1165,933,858,255,740,528,657,148,448.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2007,1259,1332,912,1079,285,865,652,859,185,672.0,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [4]:
df = parse_training_data_with_valid_topics('../data/TrainingData/1999b_TrainingData.json', topics=topics)

In [5]:
df = pd.DataFrame(df).T

In [16]:
x = df[df.columns.intersection(topics)].sum(axis=1)
x[x.isnull()]

2014b_TrainingData_52907   NaN
dtype: float64

In [3]:
_word2idx = pd.read_hdf('training_data_wv_fs_no_stopwords.hdf', '_word2idx')

In [8]:
_word2idx['zika']

146425

In [9]:
top_tokens = ['hello', 'world', 'united', 'nations']

In [11]:
topword2idx = {}
for ix, token in enumerate(top_tokens):
    topword2idx[_word2idx[token]] = ix + 1  # Start with index 1 since 0 is a special symbol

In [14]:
topword2idx[_word2idx['united']]

3